In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.DataFrame()
df['Outlook'] = [
    'sunny', 'sunny', 'overcast', 'rain', 'rain', 'rain',
    'overcast', 'sunny', 'sunny', 'rain', 'sunny',
    'overcast', 'overcast', 'rain'
]
df['Temperature'] = [
    'hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool',
    'mild', 'cool', 'mild', 'mild', 'mild', 'hot', 'mild',
]
df['Humidity'] = [
    'high', 'high', 'high', 'high', 'normal', 'normal', 'normal', 
    'high', 'normal', 'normal', 'normal', 'high', 'normal', 'high'
]
df['Windy'] = [
    'Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak',
    'Strong', 'Strong', 'Weak', 'Strong'
]
df['Decision'] = [
    'N', 'N', 'P', 'P', 'P', 'N', 'P', 'N', 'P', 'P',
    'P', 'P', 'P', 'N'
]
df

,Outlook,Temperature,Humidity,Windy,Decision
0,sunny,hot,high,Weak,N
1,sunny,hot,high,Strong,N
2,overcast,hot,high,Weak,P
3,rain,mild,high,Weak,P
4,rain,cool,normal,Weak,P
5,rain,cool,normal,Strong,N
6,overcast,cool,normal,Strong,P
7,sunny,mild,high,Weak,N
8,sunny,cool,normal,Weak,P
9,rain,mild,normal,Weak,P


In [3]:

p_p = len(df.loc[df.Decision == 'P']) / len(df)

p_n = len(df.loc[df.Decision == 'N']) / len(df)

entropy_decision = -p_n * np.log2(p_n) - p_p * np.log2(p_p)
print(f'H(S) = {entropy_decision:0.4f}')

H(S) = 0.9403


In [4]:

def f_entropy_decision(data):
    p_p = len(data.loc[data.Decision == 'P']) / len(data)
    p_n = len(data.loc[data.Decision == 'N']) / len(data)
    return -p_n * np.log2(p_n) - p_p * np.log2(p_p)


In [5]:
IG_decision_Outlook = entropy_decision # H(S)
overall_eqn = 'Gain(Decision, Outlook) = Entropy(Decision)' 

for name, Outlook in df.groupby('Outlook'):
    num_p = len(Outlook.loc[Outlook.Decision == 'P'])
    num_n = len(Outlook.loc[Outlook.Decision != 'P'])
    num_Outlook = len(Outlook)
    print(f'p(Decision=P|Outlook={name}) = {num_p}/{num_Outlook}')
    print(f'p(Decision=N|Outlook={name}) = {num_n}/{num_Outlook}')    
    print(f'p(Outlook={name}) = {num_Outlook}/{len(df)}')
    print(f'Entropy(Decision|Outlook={name}) = '\
         f'-{num_p}/{num_Outlook}.log2({num_p}/{num_Outlook}) - '\
         f'{num_n}/{num_Outlook}.log2({num_n}/{num_Outlook})')
    
    entropy_decision_outlook = 0
    
    if num_p != 0:
        entropy_decision_outlook -= (num_p / num_Outlook) \
            * np.log2(num_p / num_Outlook)
     
    if num_n != 0:
        entropy_decision_outlook -= (num_n / num_Outlook) \
            * np.log2(num_n / num_Outlook)
    
    IG_decision_Outlook -= (num_Outlook / len(df)) * entropy_decision_outlook
    print()
    overall_eqn += f' - p(Outlook={name}).'
    overall_eqn += f'Entropy(Decision|Outlook={name})'
print(overall_eqn)
print(f'Gain(Decision, Outlook) = {IG_decision_Outlook:0.4f}')


p(Decision=P|Outlook=overcast) = 4/4
p(Decision=N|Outlook=overcast) = 0/4
p(Outlook=overcast) = 4/14
Entropy(Decision|Outlook=overcast) = -4/4.log2(4/4) - 0/4.log2(0/4)

p(Decision=P|Outlook=rain) = 3/5
p(Decision=N|Outlook=rain) = 2/5
p(Outlook=rain) = 5/14
Entropy(Decision|Outlook=rain) = -3/5.log2(3/5) - 2/5.log2(2/5)

p(Decision=P|Outlook=sunny) = 2/5
p(Decision=N|Outlook=sunny) = 3/5
p(Outlook=sunny) = 5/14
Entropy(Decision|Outlook=sunny) = -2/5.log2(2/5) - 3/5.log2(3/5)

Gain(Decision, Outlook) = Entropy(Decision) - p(Outlook=overcast).Entropy(Decision|Outlook=overcast) - p(Outlook=rain).Entropy(Decision|Outlook=rain) - p(Outlook=sunny).Entropy(Decision|Outlook=sunny)
Gain(Decision, Outlook) = 0.2467


In [9]:

def IG(data, column, ent_decision=entropy_decision):
    IG_decision = ent_decision
    for name, temp in data.groupby(column):
        p_p = len(temp.loc[temp.Decision == 'P']) / len(temp)
        p_n = len(temp.loc[temp.Decision != 'P']) / len(temp)

        entropy_decision = 0

        if p_p != 0:
            entropy_decision -= (p_p) * np.log2(p_p)

        if p_n != 0:
            entropy_decision -= (p_n) * np.log2(p_n)

        IG_decision -= (len(temp) / len(df)) * entropy_decision
    return IG_decision

In [10]:

for col in df.columns[:-1]:
    print(f'Gain(Decision, {col}) = {IG(df, col):0.4f}')
    

Gain(Decision, Outlook) = 0.2467
Gain(Decision, Temperature) = 0.0292
Gain(Decision, Humidity) = 0.1518
Gain(Decision, Windy) = 0.0481


In [11]:
for name, temp in df.groupby('Outlook'):
    print('-' * 15)
    print(name)
    print('-' * 15)
    print(temp)
    print('-' * 15)

---------------
overcast
---------------
     Outlook Temperature Humidity   Windy Decision
2   overcast         hot     high    Weak        P
6   overcast        cool   normal  Strong        P
11  overcast        mild     high  Strong        P
12  overcast         hot   normal    Weak        P
---------------
---------------
rain
---------------
   Outlook Temperature Humidity   Windy Decision
3     rain        mild     high    Weak        P
4     rain        cool   normal    Weak        P
5     rain        cool   normal  Strong        N
9     rain        mild   normal    Weak        P
13    rain        mild     high  Strong        N
---------------
---------------
sunny
---------------
   Outlook Temperature Humidity   Windy Decision
0    sunny         hot     high    Weak        N
1    sunny         hot     high  Strong        N
7    sunny        mild     high    Weak        N
8    sunny        cool   normal    Weak        P
10   sunny        mild   normal  Strong        P
---------

In [12]:
df_next = df.loc[df.Outlook != 'overcast']
df_next


,Outlook,Temperature,Humidity,Windy,Decision
0,sunny,hot,high,Weak,N
1,sunny,hot,high,Strong,N
3,rain,mild,high,Weak,P
4,rain,cool,normal,Weak,P
5,rain,cool,normal,Strong,N
7,sunny,mild,high,Weak,N
8,sunny,cool,normal,Weak,P
9,rain,mild,normal,Weak,P
10,sunny,mild,normal,Strong,P
13,rain,mild,high,Strong,N


In [13]:

df_sunny = df_next.loc[df_next.Outlook == 'sunny']


In [14]:

entropy_decision = f_entropy_decision(df_sunny)
entropy_decision

0.9709505944546686

In [15]:

for col in df_sunny.columns[1:-1]:
    print(f'Gain(Decision, {col}) = {IG(df_sunny, col, entropy_decision):0.4f}')


Gain(Decision, Temperature) = 0.8281
Gain(Decision, Humidity) = 0.9710
Gain(Decision, Windy) = 0.6313


In [16]:
for name, temp in df_sunny.groupby('Humidity'):
    print('-' * 15)
    print(name)
    print('-' * 15)
    print(temp)
    print('-' * 15)


---------------
high
---------------
  Outlook Temperature Humidity   Windy Decision
0   sunny         hot     high    Weak        N
1   sunny         hot     high  Strong        N
7   sunny        mild     high    Weak        N
---------------
---------------
normal
---------------
   Outlook Temperature Humidity   Windy Decision
8    sunny        cool   normal    Weak        P
10   sunny        mild   normal  Strong        P
---------------


In [17]:

df_rain = df_next.loc[df_next.Outlook == 'rain']
entropy_decision = f_entropy_decision(df_rain)
entropy_decision

0.9709505944546686

In [18]:

for col in df_rain.columns[1:-1]:
    print(f'Gain(Decision, {col}) = {IG(df_rain, col, entropy_decision):0.4f}')


Gain(Decision, Temperature) = 0.6313
Gain(Decision, Humidity) = 0.6313
Gain(Decision, Windy) = 0.9710


In [19]:
for name, temp in df_rain.groupby('Windy'):
    print('-' * 15)
    print(name)
    print('-' * 15)
    print(temp)
    print('-' * 15)


---------------
Strong
---------------
   Outlook Temperature Humidity   Windy Decision
5     rain        cool   normal  Strong        N
13    rain        mild     high  Strong        N
---------------
---------------
Weak
---------------
  Outlook Temperature Humidity Windy Decision
3    rain        mild     high  Weak        P
4    rain        cool   normal  Weak        P
9    rain        mild   normal  Weak        P
---------------
